# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    22-Jan-2026**

**News Source / Catalyst:** "European governments turn to short-term debt as borrowing costs rise"

**Markets (s) Impacted:** FI (Sovereign bonds), Eurozone & UK

**Trade Taxonomy:** RV Macro (Curve Term Premium Expansion)

**Trade Idea (1 sentence):**
> Long a basket of short-duration (1-3y) European sovereign bonds against a short bascket of ultra-long (25y+) sovereign bonds to capture the *bear steepening* caused by waning pension demand and increased fiscal supply

**Client/Desk Pitch (30 seconds):**
> The *scarcity of Bunds* is officially over, and the structrual *forced buyers* (Pension funds) of long-dated duration are disappearing due to Dutch and UK pension reforms. Governments are responding by crowding the short end of the curve. We are enteringa a "fiscal reality" regime where long-end yields must rise significantly to attract new, non-traditional buyers. This trade stays long the "crowded" front end while shorting the "orohaned" long end.

**Why Now:**

The Dutch pension overhaul (Jan 2026) and the German Debt Breake loosening have created a supply/demand mismatch that is just beginning to be priced into the 10s30s and 2s30s spreads.

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**

The average expiry of debt is dropping because the long-end demand from penion funds has fundamentally changed. This creates a structural *duration vacuum* at the 30y point. We expect a steepening of the yield curve where long-dated yields rise faster than short-dated yields; Short-end yields are anchored by the ECB/BoE, while the long end is now market driven.

**Long Leg (Tickers + Rationale):**
- IBGS.L (iShares EUR Govt Bond 1-3yr): Benefits from government crowding and is more resilient to the supply shocks hitting the long end

**Short Leg (Tickers + Rationale):**
- IBGL.L (iShares EUR Govt Bond 15-30yr): Directly exposed to the waning demand for Dutch/UK pension funds
- EUNL.DE (iShares EUR Govt Bond 20yr+): High duration sensitivity to the "German Scarcity" reversal

**Weights:** Vol-weighted (Necessary because 30y bonds have significantly higher daily price volatility than 1y bonds)

---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |       W           | Euro-centric fiscal play; limited USD impact      |
| Oil Beta             |       M           | Higher energy prices fuel inflation, hurting the Short Leg more      |
| Rates Duration       |       S           | This is the core factor; long-end duration is the primary risk       |
| Growth vs Value      |       W           | Growth slowdown might cap yields, but supply overhang dominates       |
| Risk-On/Off          |       M           | "Fiscal Worry" usually correlates with Risk-Off       |
| China/EM Beta        |       W           | Irrelevant to European sovereign term premiums      |
| Volatility Regime    |       S           | Short Leg (Long Bonds) will see higher vol than Long Leg      |

**Catalyst Type:** Supply-Demand / Regime Shift  

**Realisation Window:** Weeks / Quarters (Long-term structural shift)

**Holding Period Selected:**  35 (Tactical window follwing Jan Dutch reforms)

**Alignment Score:** Good

---


## 4. Risk Controls & Stop Logic

**Position Sizing:** (units per leg) 4 units Long (Short-term) vs 1 unit Short (Long-term) to equalise risk

**Stop-Loss:** -1.50% (portfolio-level)  

**Take-Profit (optional):**  5.0%

**Re-entry:** Yes, if Bund 30y yields hit 3.75% and stabilise. 

**FX-Hedging Required:** No (Euro-denominated). 

**Liquidity Notes:** ADV is high on iShares UCITSL spreads are tight (<2bps) 

**Failure Mode (tick):**
- [ ] Time
- [X] Price (Global flight to quality sppresses long yields despite supply)
- [ ] Information

---


# **PYTHON IMPLEMENTATION**

In [189]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [190]:
# --- Parameters ---

long_tickers = ["IBGS.L"]     # EDIT HERE
short_tickers = ["IBGL.L", "EUNL.DE"]                    # EDIT HERE

start_date = "2024-01-01"                         # Backtest window
holding_period = 35                               # days
stop_loss = -0.015                                 # -2% stop
take_profit = 0.05                                # optional e.g. +0.03 for +3%

# --- Weightings ---

weights_long = 0.70
weights_short = np.array([0.125, 0.125])


In [191]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"]

# Drop empty cols & make sure no NA

data = data.dropna(how='all').ffill()

returns = data.pct_change().dropna()


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_41904/911793947.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"]
[*********************100%***********************]  3 of 3 completed


In [192]:
# --- Spread Returns ---

long_returns = returns[long_tickers].mean(axis=1)
short_returns = returns[short_tickers].mean(axis=1)

strategy_returns = long_returns - short_returns


In [193]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_returns.index
i = 0
n = len(strategy_returns)

while i < n - holding_period:
    pnl = 0.0
    stopped = False
    entry_date = dates[i]
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

trade_results.tail()


,Trade Date,PnL
9,2025-03-26,0.0639
10,2025-05-16,0.0027
11,2025-07-04,0.0017
12,2025-08-22,-0.0162
13,2025-10-10,-0.0054


In [194]:
# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
}

pd.Series(summary)


Total Trades         14.0000
Win Rate              0.5000
Average PnL           0.0060
Best Trade            0.0639
Worst Trade          -0.0239
Stop-Out Frequency    0.4286
dtype: float64

## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades:         14.0000
- Win Rate:              0.5000
- Average PnL:           0.0060
- Best Trade:            0.0639
- Worst Trade:          -0.0239
- Stop-Out Frequency:    0.4286

**What went right:**
- The trade successfully captures *Supply Shocks*. When the German DMO announces a shift to shorter maturities, the long-end "orphans" (IBGL.L) sell off aggressively, providing that 6%+ outlier return.

**What went wrong:**
- The "Greenland Hedge" / Flight to Quality: Whenever 2026 geopolitical tensions spike (Arctic Fissure news), investors grab long-dated Bunds as a safe haven. This causes a "Bull Flattening" that triggers your -2% stop instantly before the supply narrative can resume.
- Ticker Contamination: Using the London listing (.L) introduced FX volatility that wasn't part of my core thesis.

**Factor that actually drove results:**
- Term Premium Expansion. The results weren't driven by central bank hikes (Rates), but by the market demanding a higher "risk prize" to hold 30Y debt without the Dutch pension fund safety net.

**Would I trade this live?** (Yes / No and why)
> Yes. The trade demonstrates a robust structural edge driven by pension-fund demand withdrawal and sovereign issuance re-profiling, rather than transient central-bank policy. The positive skew, fat-tail winners, and clear stop-out clustering during exogenous risk events suggest the signal is genuine but requires regime-aware execution.

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [195]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [196]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short

basket_long
basket_short
basket_spread


Date
2024-01-02   52.7397
2024-01-03   52.3159
2024-01-04   52.4164
2024-01-05   52.2974
2024-01-08   52.2306
               ...  
2026-01-15   53.9195
2026-01-16   53.9189
2026-01-19   54.2686
2026-01-20   54.8246
2026-01-21   54.8316
Length: 526, dtype: float64

In [197]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
9,RV-782440F8,2026-01-22 07:11:53,Macro RV Spread,RV Macro Spread,IBGS.L,"IBGL.L, EUNL.DE",2025-03-26,2025-05-15,52.2032,52.7622,-0.3394,-1.5000,5.0000,CLOSED,50,
10,RV-B829F445,2026-01-22 07:11:53,Macro RV Spread,RV Macro Spread,IBGS.L,"IBGL.L, EUNL.DE",2025-05-16,2025-07-03,52.5350,54.2044,0.2660,-1.5000,5.0000,CLOSED,48,
11,RV-59D5D744,2026-01-22 07:11:53,Macro RV Spread,RV Macro Spread,IBGS.L,"IBGL.L, EUNL.DE",2025-07-04,2025-08-21,54.5124,54.7388,0.1692,-1.5000,5.0000,CLOSED,48,
12,RV-1C671CAC,2026-01-22 07:11:53,Macro RV Spread,RV Macro Spread,IBGS.L,"IBGL.L, EUNL.DE",2025-08-22,2025-09-11,54.6565,54.2777,-1.6167,-1.5000,5.0000,STOPPED,20,
13,RV-D971A774,2026-01-22 07:11:53,Macro RV Spread,RV Macro Spread,IBGS.L,"IBGL.L, EUNL.DE",2025-10-10,2025-11-28,54.7023,54.7916,-0.5357,-1.5000,5.0000,CLOSED,49,


In [198]:
blotter.to_csv("trade_blotter.csv", index=False)